In [ ]:
from crowd_evaluation import ConfidenceEvaluatorNew
from datasets import EntailmentDataset
import numpy as np
import plotly.graph_objects as go

In [ ]:
dataset = EntailmentDataset('/Users/julianassmann/Documents/Bachelor-Thesis/Code/datasets/ent_tem/rte.standardized.tsv',
                            prefilter_mode='truth',
                            debug=True)
evalc = ConfidenceEvaluatorNew(dataset, debug=True)

In [ ]:
ps, confs = evalc.evaluate_workers_with_confidence(dataset.workers, 0.95, min_shared_samples=10)

In [ ]:
ps, confs = np.minimum(ps, 1), np.minimum(confs, .5)

In [ ]:
ind = dataset.workers
min = 0
max = 10
x_labels = np.char.add(np.full(dataset.num_workers, "W"), np.char.mod('%d', ind + 1))
fig = go.Figure()
fig.add_trace(go.Bar(y=dataset.get_measured_error_rates_for_workers(dataset.workers[min:max])*100, name=r'$p_{true}$', x=x_labels))
# fig.add_trace(go.Bar(y=ps*100, name=r'$p_{est}$', x=x_labels))
fig.add_trace(go.Bar(y=ps[min:max]*100, name=r'$p_{est}$', error_y=dict(type='data', array=confs[min:max]*100), x=x_labels))
fig.update_layout(barmode='group', title='Measured and estimated error rates in %')
fig.show()

In [ ]:
accuracies = []
confidence_levels = np.arange(5, 100, step=5) / 100
for c in confidence_levels:
    ps, confs = evalc.evaluate_workers_with_confidence(dataset.workers, c, min_shared_samples=10)
    p_measured = dataset.get_measured_error_rates_for_workers(dataset.workers)
    min, max = ps-confs, ps+confs

    accuracy = np.count_nonzero(np.where((ps <= max) & (ps >= min), 1, 0)) / len(ps)
    print(c, ':', accuracy)
    accuracies.append(accuracy)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(confidence_levels, accuracies, label='Evaluator')
plt.plot(confidence_levels, confidence_levels, label='optimal')
plt.xlabel('confidence level')
plt.ylabel('accuracy')
plt.legend()
plt.show()